# Pytorch Tutorial

## PyTorch Background

In [ ]:
import torch

data = [[1,2,3], [4,5,6], [7,8,9]]
data_tensor = torch.tensor(data)
print(data_tensor)

In [ ]:
ones_tensor = torch.ones(size=data_tensor.shape, dtype=int)
print(ones_tensor)

# these tensors behave almost exactly like numpy arrays
print(ones_tensor @ data_tensor)

## Datasets

In [ ]:
import torch
import numpy
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

print(training_data)
print(test_data)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

In [ ]:
import os
import numpy as np
import pandas as pd
from torchvision.io import read_image

class NoiseDataset(Dataset):
    def __init__(
        self,
        csv_file="TrainingDataSet.csv",
        root_dir_noisy="TrainingDataSet",
        root_dir_ref="./",
        transform=None,
    ):
        self.name_csv = pd.read_csv(csv_file)
        self.root_dir_noisy = root_dir_noisy
        self.root_dir_ref = root_dir_ref
        self.transform = transform

    def __len__(self):
        return len(self.name_csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        ref_img_name = os.path.join(self.root_dir_ref, self.name_csv.iloc[idx, 0])
        noisy_img_name = os.path.join(self.root_dir_noisy, self.name_csv.iloc[idx, 2])
        ref_image = read_image(ref_img_name)
        noisy_image = read_image(noisy_img_name)

        if self.transform:
            ref_image = self.transform(ref_image)
            noisy_image = self.transform(noisy_image)
        return noisy_image, ref_image

In [ ]:
cats_dataset = NoiseDataset(
            csv_file="data/cats/training.csv",
            root_dir_noisy="data/cats/training",
        )

print(f"len(cats_dataset): {len(cats_dataset)}")

cats_dataloader = DataLoader(cats_dataset, batch_size=3, shuffle=True)

noisy_images, ref_images = next(iter(cats_dataloader))
print(f"noisy_images.shape: {noisy_images.shape}")

In [ ]:
figure = plt.figure(figsize=(6, 8))
rows = noisy_images.shape[0]
for i in range(0, rows):
    figure.add_subplot(rows, 2, (2 * i) + 1)
    plt.title("Noisy")
    plt.axis("off")
    plt.imshow(noisy_images[i].squeeze(), cmap="gray")

    figure.add_subplot(rows, 2, (2 * i) + 2)
    plt.title("Reference")
    plt.axis("off")
    plt.imshow(ref_images[i].squeeze(), cmap="gray")

## Transforms

In [ ]:
from torchvision.transforms import Resize

cats_dataset = NoiseDataset(
            csv_file="data/cats/training.csv",
            root_dir_noisy="data/cats/training",
            transform=Resize((32, 32))
        )

cats_dataloader = DataLoader(cats_dataset, batch_size=3, shuffle=True)

noisy_images, ref_images = next(iter(cats_dataloader))

figure = plt.figure(figsize=(6, 8))
rows = noisy_images.shape[0]
for i in range(0, rows):
    figure.add_subplot(rows, 2, (2 * i) + 1)
    plt.title("Noisy")
    plt.axis("off")
    plt.imshow(noisy_images[i].squeeze(), cmap="gray")

    figure.add_subplot(rows, 2, (2 * i) + 2)
    plt.title("Reference")
    plt.axis("off")
    plt.imshow(ref_images[i].squeeze(), cmap="gray")

## Network Architecture

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # input images are 28px by 28px
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10), # there are 10 output classes
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

## Optimization and Hyperparameters 

In [ ]:
# set hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 5

# initialize loss function
loss_fn = nn.CrossEntropyLoss()

# initialize optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Training and Testing

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 200 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

## Saving the model

In [ ]:
# save model weights
torch.save(model.state_dict(), "model-weights.pth")
model2 = NeuralNetwork()
model2.load_state_dict(torch.load("model-weights.pth"))

# save model object
torch.save(model, 'model.pth')
model3 = torch.load('model.pth')